In [1]:
from pyspark.sql import SparkSession
import time
import os

In [8]:
spark = (SparkSession.builder
    .master("local")
    .appName("SQLite JDBC")
    .config(
        "spark.jars",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config(
        "spark.driver.extraClassPath",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config("spark.executor.cores", '8')
    .config("spark.sql.compute.ops_on_diff_frames", True) 
    .getOrCreate())

24/04/29 16:51:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
import pyspark.pandas as ps

con="jdbc:sqlite:{}/database.sqlite".format(os.getcwd())

/home/marinevas/.asdf/installs/python/3.10.13/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [20]:
rankings = ps.read_sql("rankings", con=con)
# rankings = rankings.rename(columns={'player': 'player_id'})
rankings.head(2)

,ranking_date,rank,player,points
0,20000110,1,101736,4135.0
1,20000110,2,102338,2915.0


In [21]:
players = ps.read_sql("players", con=con)
players.head(2)

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581


In [34]:
matches = ps.read_sql('matches', con=con)
matches.head(2)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,winner1_id,winner2_id,loser1_id,loser2_id,winner1_name,winner1_hand,winner1_ht,winner1_ioc,winner1_age,winner2_name,winner2_hand,winner2_ht,winner2_ioc,winner2_age,loser1_name,loser1_hand,loser1_ht,loser1_ioc,loser1_age,loser2_name,loser2_hand,loser2_ht,loser2_ioc,loser2_age,winner1_rank,winner1_rank_points,winner2_rank,winner2_rank_points,loser1_rank,loser1_rank_points,loser2_rank,loser2_rank_points
0,2017-M020,Brisbane,Hard,32,A,20170102.0,271,104678.0,None,None,Viktor Troicki,R,193.0,SRB,30.8,106415.0,None,Q,Yoshihito Nishioka,L,170.0,JPN,21.2,6-4 7-5,3,R32,91.0,11.0,5.0,64.0,45.0,35.0,6.0,11.0,1.0,3.0,0.0,1.0,82.0,53.0,33.0,13.0,11.0,6.0,10.0,29.0,1385.0,100.0,604.0,NaN,None,None,NaN,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-M020,Brisbane,Hard,32,A,20170102.0,272,106378.0,None,None,Kyle Edmund,R,188.0,GBR,21.9,124014.0,None,Q,Ernesto Escobedo,R,185.0,USA,20.4,7-6(4) 7-6(6),3,R32,130.0,11.0,2.0,83.0,48.0,37.0,19.0,12.0,2.0,3.0,11.0,3.0,113.0,67.0,39.0,27.0,12.0,9.0,10.0,45.0,1001.0,141.0,443.0,NaN,None,None,NaN,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
rankings = rankings.to_spark()
players = players.to_spark()
matches = matches.to_spark()

/home/marinevas/.asdf/installs/python/3.10.13/lib/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


# heavy query

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize SparkSession
spark = (SparkSession.builder
    .master("local")
    .appName("SQLite JDBC")
    .config(
        "spark.jars",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config(
        "spark.driver.extraClassPath",
        "{}/sqlite-jdbc-3.34.0.jar".format(os.getcwd()))
    .config("spark.executor.cores", '8')
    .config("spark.sql.compute.ops_on_diff_frames", True) 
    .getOrCreate())

start_time = time.time()

# Load the rankings data into a DataFrame (replace "path_to_rankings" with the actual file path)
url = "jdbc:sqlite:{}/database.sqlite".format(os.getcwd())
rankings_df = spark.read.jdbc(url=url, table="rankings")

rankings_df.repartition(7)

# Self-join the rankings DataFrame
rankings_df = rankings_df.alias("r1") \
    .join(rankings_df.alias("r2"), col("r1.ranking_date") < col("r2.ranking_date")) \
    .select(col("r1.ranking_date").alias("date1"),
            col("r1.rank").alias("rank1"),
            col("r1.points").alias("points1"),
            col("r2.ranking_date").alias("date2"),
            col("r2.rank").alias("rank2"),
            col("r2.points").alias("points2"))

print(time.time() - start_time)

spark.stop() 